# AgentCore Gateway インターセプター - ヘッダー伝播ソリューション

## 概要

多くの企業は、MCP サーバーを通じて下流 API にアクセスする AI エージェントシステムを構築する際、重大なセキュリティ課題に直面しています。核心的な問題は、エージェントがツールを呼び出し、そのツールが続けて下流 API を呼び出すマルチホップワークフローにおける、安全なトークン交換とアイデンティティ伝播です。

企業は、MCP サーバーから下流 API へスコープを絞ったトークンと最小権限の認証情報を送信するためにトークンを交換する必要があります。これには、インバウンド JWT トークンからプリンシパルのアイデンティティとメタデータを抽出し、呼び出し元の認証情報に基づいてきめ細かいアクセス制御を実行し、特定の下流 API 呼び出しに対して適切にスコープされた認証情報を取得するためにトークンを動的に交換する必要があります。

組織は元の認可トークンを単純に下流サービスに転送することはできません。下流サービスが侵害された場合、元のトークンが盗まれて悪用される可能性があります。下流サービスは特定の操作に意図された以上の広い権限を持つ可能性があり、権限昇格のリスクを生み出します。混乱した代理（Confused Deputy）問題は、サービスが適切なコンテキスト検証なしにユーザーの代わりに動作する際に発生し、適切な監査証跡なしでは、どのサービスがどのユーザーの代わりにどのリソースにアクセスしたかを追跡することが困難になります。この問題は、異なるテナントや組織の異なるユーザーが同じ AgentCore Gateway にアクセスするマルチテナント MCP サーバー環境で深刻化します。これらの環境では、テナント間の厳格なデータ分離が必要であり、それぞれが異なる下流 API アクセスパターンと権限を持ちます。代理パターンはすべてのテナントに渡って一貫して実施される必要があります。

インバウンドトークンを中間レイヤーに渡すメカニズムがなければ、企業はトークンの真正性を検証したり、トークン交換前にきめ細かいアクセス制御ポリシーを適用したり、特定の下流 API 呼び出しに対して適切にスコープされたトークンを生成したり、マルチホップワークフロー全体を通じて適切な実行コンテキストを維持したり、共有インフラストラクチャでのテナント分離を確保したりすることができません。

AgentCore Gateway は、Gateway インターセプターと AWS Lambda ターゲットを通じてこれに対処します。AWS Lambda ターゲット統合と Gateway インターセプターの両方が、下流 API を呼び出す前にトークン交換を実行する抽象化レイヤーとして機能します。この例では、Gateway インターセプターを使用してインバウンドトークンをカスタムヘッダー経由で AWS Lambda ターゲット関数に渡し、トークン署名検証と有効期限チェックを含む有効性チェックを実行し、抽出されたプリンシパルに基づいてきめ細かいアクセス制御を適用し、下流 API に固有の狭くスコープされたトークンにトークンを交換し、新しく交換された最小権限トークンで下流 API を呼び出す方法を示します。

このソリューションにより、各ホップが個別のスコープされたトークンを取得する安全な代理パターン、ワークフロー全体を通じてユーザーアイデンティティを維持する JWT ベースの実行コンテキスト伝播、適切なデータ分離のためのテナント分離、コンプライアンスのための明確な監査証跡、および認証を個別に処理しながら MCP スキーマを維持する分離されたセキュリティが可能になります。

### チュートリアル詳細


| 情報                 | 詳細                                                      |
|:---------------------|:----------------------------------------------------------|
| チュートリアルタイプ | インタラクティブ                                          |
| AgentCore コンポーネント | AgentCore Gateway                                     |
| Gateway ターゲットタイプ | AWS Lambda                                            |
| インバウンド認証     | OAuth                                                     |
| アウトバウンド認証   | AWS IAM                                                   |
| チュートリアルコンポーネント | AgentCore Gateway の作成と AgentCore Gateway の呼び出し |
| チュートリアル分野   | クロスバーティカル                                        |
| 難易度               | 簡単                                                      |
| 使用 SDK             | boto3                                                     |

チュートリアルの最初の部分では、Lambda 用の AgentCore Gateway ターゲットを作成します

### チュートリアルアーキテクチャ

![アーキテクチャ図](images/04-gateway-interceptor-header-propagation.png)

---

1. **クライアント**: 認証トークンを使用して Lambda ベースの MCP サーバーへのリクエストを開始
2. **AgentCore Gateway**: クライアントリクエストをインターセプターを通じて処理のためにルーティング
3. **Gateway インターセプター**: クライアントワークフローのトークン交換とメタデータ処理を実行
4. **ターゲット Lambda (MCP サーバー)**: 交換された認証情報で処理済みリクエストを受信

## ステップ 1: 依存関係のインストール

このステップでは、MCP ゲートウェイを作成するために AWS サービスと対話するために必要な boto3 パッケージをインストールします。

In [ ]:
import subprocess
import sys

# 必要なパッケージをインストール
subprocess.check_call([sys.executable, "-m", "pip", "install", "boto3", "requests"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "strands-agents"])
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "setuptools", "pip"])

print("✅ 依存関係のインストールが完了しました")

## ステップ 2: AWS クライアントと設定の初期化

このステップでは、必要な AWS サービスクライアントをセットアップし、競合を避けるためにリソース命名用の一意のタイムスタンプを作成します。

In [ ]:
import boto3
import json
import time
import zipfile
import io
from datetime import datetime

timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
lambda_client = boto3.client('lambda', region_name='us-east-1')
iam_client = boto3.client('iam', region_name='us-east-1')
agentcore_client = boto3.client('bedrock-agentcore-control', region_name='us-east-1')

## ステップ 3: AgentCore Gateway インターセプター関数の作成

このステップでは、以下を行う AgentCore Gateway インターセプター関数を作成します：
- IAM ロールの作成: AgentCore Gateway インターセプター実行のための適切な権限をセットアップ
- ヘッダーロジックの実装: Authorization ヘッダーをリクエストヘッダーからリクエストボディに移動
- MCP プロトコルの処理: MCP ゲートウェイリクエストを処理し、変換されたレスポンスを返却

AgentCore Gateway インターセプター関数は Authorization ヘッダーを抽出し、リクエストボディの引数に配置することで、下流処理で利用可能にします。

In [ ]:
def create_interceptor_lambda_complete():
    """AgentCore Gateway インターセプターを IAM ロールと関数付きで完全に作成"""
    lambda_client = boto3.client('lambda', region_name='us-east-1')
    iam_client = boto3.client('iam', region_name='us-east-1')
    
    # Lambda 用の IAM ロールを作成
    lambda_trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    lambda_role_name = f'InterceptorLambdaRole-{timestamp}'
    
    lambda_role_response = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=json.dumps(lambda_trust_policy),
        Description='IAM role for Interceptor Lambda'
    )
    
    # 基本的な Lambda 実行ポリシーをアタッチ
    iam_client.attach_role_policy(
        RoleName=lambda_role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    
    lambda_role_arn = lambda_role_response['Role']['Arn']
    print(f"Lambda IAM ロールを作成しました: {lambda_role_arn}")
    
    # ロールが利用可能になるまで待機
    print("Lambda ロールが利用可能になるまで待機中...")
    time.sleep(10)
    
    # トークン交換用の認可トークンを処理する Lambda 関数コード
    lambda_code = '''
import json
import uuid

def lambda_handler(event, context):
    # 正しい構造から Gateway リクエストを抽出
    mcp_data = event.get('mcp', {})
    gateway_request = mcp_data.get('gatewayRequest', {})
    headers = gateway_request.get('headers', {})
    body = gateway_request.get('body', {})
    extended_body = body
    
    # トークン交換用の認可トークンを抽出
    # 注意: この認可トークンは伝播用ではなく、トークン交換用です
    auth_header = headers.get('authorization', '') or headers.get('Authorization', '')
    
    # 伝播用のカスタムヘッダーを抽出
    custom_header = headers.get('customHeaderKey', '')
    
    # 認可トークンを使用したトークン交換のコードはここに記述
    # 受信したトークンは交換目的で使用され、なりすましには使用されません
    
    if "params" in extended_body and "arguments" in extended_body["params"]:
        # 交換されたトークンまたは認証情報を引数に追加（元の認証トークンではなく）
        extended_body["params"]["arguments"]["exchanged_credentials"] = "exchanged_token_placeholder"
        # 下流処理用にカスタムヘッダーを引数に追加
        extended_body["params"]["arguments"]["customHeaderKey"] = custom_header
    
    # 元の認可ヘッダーを渡さずに変換されたリクエストを返す
    response = {
        "interceptorOutputVersion": "1.0",
        "mcp": {
            "transformedGatewayRequest": {
                "headers": {
                    "Accept": "application/json",
                    "Content-Type": "application/json"
                },
                "body": extended_body
            }
        }
    }
    return response
'''
    
    # Lambda 用の ZIP ファイルを作成
    zip_buffer = io.BytesIO()
    with zipfile.ZipFile(zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        zip_file.writestr('lambda_function.py', lambda_code)
    
    zip_buffer.seek(0)
    
    # Lambda 関数を作成
    lambda_function_name = f'interceptor-lambda-{timestamp}'
    
    lambda_response = lambda_client.create_function(
        FunctionName=lambda_function_name,
        Runtime='python3.13',
        Role=lambda_role_arn,
        Handler='lambda_function.lambda_handler',
        Code={
            'ZipFile': zip_buffer.read()
        },
        Description='Interceptor Lambda for MCP Gateway'
    )
    
    lambda_arn = lambda_response['FunctionArn']
    print(f"Lambda 関数を作成しました: {lambda_arn}")
    
    return lambda_arn

# Lambda 関数を作成
lambda_arn = create_interceptor_lambda_complete()
print(f"\n✅ Lambda の作成が完了しました: {lambda_arn}")

## ステップ 4: インターセプター設定を持つ MCP Gateway の作成

このステップでは、以下を行う MCP Gateway を作成します：
- Gateway IAM ロールの作成: ゲートウェイサービスの権限をセットアップ
- インターセプターの設定: Lambda 関数をリクエストインターセプターとしてリンク
- 認証のセットアップ: JWT ベースの認証を設定
- Ready 状態の待機: ゲートウェイが完全に稼働するまでポーリング

ゲートウェイはすべての受信リクエストをインターセプトし、ターゲットに転送する前に Lambda 関数を通じてルーティングします。

In [ ]:
# Gateway の作成
iam_response = iam_client.create_role(
    RoleName=f'BedrockAgentCoreGatewayRole-{timestamp}',
    AssumeRolePolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {"Effect": "Allow", "Principal": {"Service": "bedrock-agentcore.amazonaws.com"}, "Action": "sts:AssumeRole"}
        ]
    })
)

iam_client.put_role_policy(
    RoleName=f'BedrockAgentCoreGatewayRole-{timestamp}',
    PolicyName='LambdaInvokePolicy',
    PolicyDocument=json.dumps({
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": [
                    "lambda:InvokeAsync",
                    "lambda:InvokeFunction"
                ],
                "Resource": "*"
            }
        ]
    })
)

gateway_response = agentcore_client.create_gateway(
    name=f"interceptor-lambda-gateway-{timestamp}",
    protocolType="MCP",
    protocolConfiguration={"mcp": {"supportedVersions": ["2025-03-26", "2025-06-18"]}},
    interceptorConfigurations=[{
        "interceptor": {"lambda": {"arn": lambda_arn}},
        "interceptionPoints": ["REQUEST"],
        "inputConfiguration": {"passRequestHeaders": True}
    }],
    authorizerType="CUSTOM_JWT",
    authorizerConfiguration={
        "customJWTAuthorizer": {
            "discoveryUrl": "DISCOVERY_URL",
            "allowedClients": ["CLIENT_ID"]
        }
    },
    roleArn=iam_response['Role']['Arn']
)

gateway_id = gateway_response['gatewayId']

# Gateway が作成されたことを確認
gateway_info = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)

while True:
    status_response = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
    if status_response.get('status') == 'READY':
        gateway_url = status_response.get('gatewayUrl')
        break
    time.sleep(10)
print(f"Gateway を作成しました。ステータス: {gateway_info.get('status')}")
print(f"✅ Gateway: {gateway_url}")

## ステップ 5: Gateway ステータスの確認

このステップでは、ゲートウェイが適切に設定されリクエストを処理する準備ができていることを確認するための追加検証チェックを実行します。

In [ ]:
# Gateway を再度取得
gateway_info_check = agentcore_client.get_gateway(gatewayIdentifier=gateway_id)
print(f"Gateway 確認: {gateway_info_check}")

## ステップ 6: ターゲット Lambda 関数と Gateway ターゲットの作成

このステップでは、以下を行うターゲット Lambda 関数を作成します：
- ターゲット Lambda の作成: 処理済みリクエストを受信するシンプルな Lambda
- IAM ロールのセットアップ: ターゲット Lambda の適切な権限
- Gateway ターゲットの作成: Lambda をツールスキーマ付きの MCP ターゲットとして登録
- ツールスキーマの定義: 必須パラメータを含む MCP ツールインターフェースを指定

ターゲット Lambda は、インターセプターによって処理されたリクエストを受信します。Authorization ヘッダーはリクエストボディに移動されています。

In [ ]:
def create_gateway_lambda_target_complete(gateway_id, gateway_url):
    """Gateway Lambda ターゲットを作成し、準備完了まで待機"""
    lambda_client = boto3.client('lambda', region_name='us-east-1')
    iam_client = boto3.client('iam', region_name='us-east-1')
    
    # ターゲット用のシンプルな Lambda 関数を作成
    target_lambda_code = '''import json

def lambda_handler(event, context):
    # 交換された認証情報を処理するコードはここに記述
    # 受信したリクエストには元の認可トークンではなく、交換されたトークンが含まれます
    
    # カスタムヘッダー値をイベントから直接抽出
    custom_header_value = event.get("customHeaderKey", "")
    
    # カスタムヘッダー値のみをログ出力
    print(f"Custom Header Value: {custom_header_value}")
    
    # MCP デモツール用のシンプルなレスポンス
    response = {
        "jsonrpc": "2.0",
        "id": event.get("id", 1),
        "result": {
            "message": "Hello from Lambda target!",
            "customHeaderKey": custom_header_value
        }
    }
    
    return response
'''
    
    # ターゲット Lambda 用の IAM ロールを作成
    target_lambda_trust_policy = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }
    
    target_lambda_role_name = f'TargetLambdaRole-{timestamp}'
    
    target_lambda_role_response = iam_client.create_role(
        RoleName=target_lambda_role_name,
        AssumeRolePolicyDocument=json.dumps(target_lambda_trust_policy),
        Description='IAM role for Target Lambda'
    )
    
    # 基本的な Lambda 実行ポリシーをアタッチ
    iam_client.attach_role_policy(
        RoleName=target_lambda_role_name,
        PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
    )
    
    target_lambda_role_arn = target_lambda_role_response['Role']['Arn']
    print(f"ターゲット Lambda IAM ロールを作成しました: {target_lambda_role_arn}")
    
    # ターゲット Lambda ロールが利用可能になるまで待機
    print("ターゲット Lambda ロールが利用可能になるまで待機中...")
    time.sleep(10)
    
    # ターゲット Lambda 用の ZIP ファイルを作成
    target_zip_buffer = io.BytesIO()
    with zipfile.ZipFile(target_zip_buffer, 'w', zipfile.ZIP_DEFLATED) as zip_file:
        zip_file.writestr('lambda_function.py', target_lambda_code)
    
    target_zip_buffer.seek(0)
    
    # ターゲット Lambda 関数を作成
    target_lambda_function_name = f'target-lambda-{timestamp}'
    
    target_lambda_response = lambda_client.create_function(
        FunctionName=target_lambda_function_name,
        Runtime='python3.13',
        Role=target_lambda_role_arn,
        Handler='lambda_function.lambda_handler',
        Code={
            'ZipFile': target_zip_buffer.read()
        },
        Description='Target Lambda for MCP Gateway'
    )
    
    target_lambda_arn = target_lambda_response['FunctionArn']
    print(f"ターゲット Lambda 関数を作成しました: {target_lambda_arn}")
    target_name = f"lambda-target-{timestamp}"
    
    target_response = agentcore_client.create_gateway_target(
        gatewayIdentifier=gateway_id,
        name=target_name,
        targetConfiguration={
            "mcp": {
                "lambda": {
                    "lambdaArn": target_lambda_response['FunctionArn'],
                    "toolSchema": {
                        "inlinePayload": [{
                            "description": "Tool to execute demo MCP",
                            "inputSchema": {
                                "properties": {
                                    "key1": {"description": "key1 for demo target", "type": "string"},
                                    "key2": {"description": "key2 for demo target", "type": "string"},
                                    "key3": {"description": "key3 for demo target", "type": "string"}
                                },
                                "required": ["key1"],
                                "type": "object"
                            },
                            "name": "mcp_demo"
                        }]
                    }
                }
            }
        },
        credentialProviderConfigurations=[{"credentialProviderType": "GATEWAY_IAM_ROLE"}]
    )
    
    target_id = target_response['targetId']
    
    while True:
        status_response = agentcore_client.get_gateway_target(gatewayIdentifier=gateway_id, targetId=target_id)
        if status_response.get('status') == 'READY':
            break
        time.sleep(10)
    
    print(f"✅ ターゲット: {target_id}")
    
    return target_id, target_lambda_arn, target_name

# Gateway Lambda ターゲットを作成
lambda_target_id, target_lambda_arn, target_name = create_gateway_lambda_target_complete(gateway_id, gateway_url)
print(f"\n✅ Lambda ターゲットの作成が完了しました: {lambda_target_id}")

## ステップ 7: サマリーとリソース情報

このステップでは、作成されたすべてのリソースのサマリーを表示します：
- タイムスタンプ: このデプロイメントの一意の識別子
- Lambda ARN: AgentCore Gateway インターセプター関数
- Gateway ID: MCP Gateway の識別子
- Gateway URL: MCP クライアント接続用のエンドポイント
- Target ID: ターゲット Lambda 設定

Gateway URL を使用して MCP クライアントを接続し、ヘッダー伝播機能をテストします。

In [ ]:
print(f"\n🎉 完了")
print(f"📅 タイムスタンプ: {timestamp}")
print(f"🔧 Lambda: {lambda_arn}")
print(f"🚪 Gateway: {gateway_id}")
print(f"🌐 URL: {gateway_url}")
print(f"🎯 ターゲット: {lambda_target_id}")

## ステップ 8: Cognito 認証を使用した MCP Gateway のテスト

このステップでは、以下の方法を示します：
- Cognito トークンの取得: ゲートウェイ設定で使用される Cognito エンドポイントで認証
- MCP エンドポイントの呼び出し: ベアラートークンを使用してゲートウェイに認証済みリクエストを行う
- ヘッダー伝播のテスト: AgentCore Gateway インターセプターが Authorization ヘッダーを適切に処理することを検証

リクエストは MCP プロトコル形式を使用し、ターゲットスキーマで定義された `mcp_demo` ツールを呼び出します。

In [ ]:
import requests
import base64

# Cognito 設定
client_id = "CLIENT_ID"  # Gateway 認証設定から
client_secret = "CLIENT_SECRET"  # 実際のシークレットに置き換え
token_endpoint = "TOKEN_ENDPOINT"

# Cognito トークンを取得
auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()

token_response = requests.post(
    token_endpoint,
    headers={
        "Authorization": f"Basic {auth_header}",
        "Content-Type": "application/x-www-form-urlencoded"
    },
    data="grant_type=client_credentials"
)

if token_response.status_code == 200:
    access_token = token_response.json()["access_token"]
    print(f"✅ トークンを取得しました: {access_token[:20]}...")
    
    # まず利用可能なツールを一覧表示
    list_tools_body = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/list"
    }
    
    list_response = requests.post(
        gateway_url,
        headers={
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json",
            "customHeaderKey": "custom-metadata-value"
        },
        json=list_tools_body
    )
    
    print(f"\n📋 ツール一覧レスポンスステータス: {list_response.status_code}")
    print(f"📋 利用可能なツール: {list_response.text}")
    
    # 一覧レスポンスからツール名を抽出
    tool_name = "mcp_demo"  # デフォルトのフォールバック
    if list_response.status_code == 200:
        try:
            tools_data = list_response.json()
            if "result" in tools_data and "tools" in tools_data["result"]:
                tools = tools_data["result"]["tools"]
                if tools and len(tools) > 0:
                    tool_name = tools[0]["name"]
                    print(f"📋 使用するツール: {tool_name}")
        except Exception as e:
            print(f"⚠️ ツール一覧をパースできませんでした。デフォルトを使用: {e}")
    
    # MCP リクエストボディ
    mcp_body = {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "tools/call",
        "params": {
            "name": tool_name,
            "arguments": {
                "key1": "test_value_1",
                "key2": "test_value_2",
                "key3": "test_value_3"
            }
        }
    }
    
    # MCP Gateway を呼び出し
    mcp_response = requests.post(
        gateway_url,
        headers={
            "Authorization": f"Bearer {access_token}",
            "Content-Type": "application/json",
            "customHeaderKey": "custom-metadata-value"
        },
        json=mcp_body
    )
    
    print(f"\n📡 MCP レスポンスステータス: {mcp_response.status_code}")
    print(f"📄 MCP レスポンス: {mcp_response.text}")
    
else:
    print(f"❌ トークンリクエストに失敗しました: {token_response.status_code}")
    print(f"エラー: {token_response.text}")

## ステップ 9: Strands Agent との統合

次に、認証をサポートする Strands エージェントと統合しましょう。

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent
import logging

# ロギングを設定
logging.getLogger("strands").setLevel(logging.INFO)
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s",
    handlers=[logging.StreamHandler()]
)

def create_streamable_http_transport():
    """OAuth トークン付きのトランスポートを作成"""
    return streamablehttp_client(
        gateway_url,
        headers={"Authorization": f"Bearer {access_token}"}
    )

client = MCPClient(create_streamable_http_transport)

# Bedrock モデルを作成
model = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

print("✅ 認証付きの Strands エージェントを設定しました")

In [ ]:
with client:
    # 利用可能なツールを一覧表示
    tools = client.list_tools_sync()
    
    # エージェントを作成
    agent = Agent(model=model, tools=tools)
    
    print(f"読み込まれたツール: {agent.tool_names}\n")
    
    # テスト 1: ツールを一覧表示
    print("テスト 1: 利用可能なツールを一覧表示")
    print("=" * 50)
    response = agent("こんにちは、利用可能なすべてのツールを一覧表示してください")
    print(f"エージェントレスポンス: {response}\n")

In [ ]:
with client:
    # 利用可能なツールを一覧表示
    tools = client.list_tools_sync()
    
    # エージェントを作成
    agent = Agent(model=model, tools=tools)
    
    print(f"読み込まれたツール: {agent.tool_names}\n")
    
    # テスト: 直接ツール呼び出し
    print("テスト: 直接ツール呼び出し")
    print("=" * 50)
    result = client.call_tool_sync(
        tool_use_id="get-order-123",
        name=f"{target_name}___mcp_demo",
        arguments={"key1": "test"}
    )
    print(f"ツール結果: {result['content'][0]['text']}")